### CONTENT

- **[SQL Environment Setup](#sqlsetup)**
- **[PART 1](#part1)**
  - **[Q1](#q1)**
  - **[Q2](#q2)**
  - **[Q3](#q3)**
  - **[Q4](#q4)**
  - **[Q5](#q5)**
  - **[Q6](#q6)**
  - **[Q7](#q7)**
  - **[Q8](#q8)**
  - **[Q9](#q9)**
- **[PART 2](#part2)**
  - **[Q10](#q10)**
  - **[Q11](#q11)**
  - **[Q12](#q12)**
  - **[Q13](#q13)**

### Create the SQL Environment <a id="sqlsetup"></a>

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('sqlite:///downloads/SQLFiles Tier 2/sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


### Part 1 <a id = 'part1'></a>

### Q1: <a id = 'q1'></a>
Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [3]:
con = engine.connect()
rs = con.execute('''
    SELECT *
    FROM Facilities
    WHERE membercost > 0
    ORDER BY membercost ASC
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys() # Add the column names
con.close()

display(df)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,6,Squash Court,3.5,17.5,5000,80
1,0,Tennis Court 1,5.0,25.0,10000,200
2,1,Tennis Court 2,5.0,25.0,8000,200
3,4,Massage Room 1,9.9,80.0,4000,3000
4,5,Massage Room 2,9.9,80.0,4000,3000


### Q2: <a id='q2'></a>
How many facilities do not charge a fee to members?

In [4]:
con = engine.connect()
rs = con.execute('''
    SELECT COUNT(facid) AS f
    FROM Facilities
    WHERE membercost = 0
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

print('The number of facilities that does not charge a fee to members is '+ str(int(df['f']))+'.')

The number of facilities that does not charge a fee to members is 4.


### Q3: <a id='q3'></a>
Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [5]:
con = engine.connect()
rs = con.execute('''
    SELECT facid
        , name
        , membercost
        , monthlymaintenance
    FROM Facilities
    WHERE membercost > 0 AND
        membercost/monthlymaintenance < .2
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

display(df)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


### Q4: <a id = 'q4'></a>
Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [6]:
con = engine.connect()
rs = con.execute('''
    SELECT *
    FROM Facilities
    WHERE facid IN (1,5)
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

display(df)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


### Q5: <a id = 'q5'></a>
Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100 dollars. Return the name and monthly maintenance of the facilities in question.

In [7]:
con = engine.connect()
rs = con.execute('''
    SELECT name
        , monthlymaintenance
        , CASE WHEN monthlymaintenance > 100 THEN 'expensive'
          ELSE 'cheap' END AS cost
    FROM Facilities
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

display(df)

,name,monthlymaintenance,cost
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


## Q6: <a id = 'q6'></a>
You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [8]:
con = engine.connect()
rs = con.execute('''
    SELECT firstname AS First
        , surname AS Last
    FROM Members
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

display(df.transpose())

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
First,GUEST,Darren,Tracy,Tim,Janice,Gerald,Burton,Nancy,Tim,Ponder,...,Ramnaresh,Douglas,Henrietta,David,Henry,Millicent,Hyacinth,John,Erica,Darren
Last,GUEST,Smith,Smith,Rownam,Joplette,Butters,Tracy,Dare,Boothe,Stibbons,...,Sarwin,Jones,Rumney,Farrell,Worthington-Smyth,Purview,Tupperware,Hunt,Crumpet,Smith


### Q7: <a id = 'q7'></a> 
Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [9]:
con = engine.connect()
rs = con.execute('''
    SELECT firstname || ' ' || surname AS member
        , name AS court
    FROM Bookings
    INNER JOIN Facilities
        ON Bookings.facid = Facilities.facid
    INNER JOIN Members
        ON Bookings.memid = Members.memid
    WHERE name IN ('Tennis Court 1', 'Tennis Court 2')
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

display(df)

,member,court
0,Tracy Smith,Tennis Court 1
1,Tracy Smith,Tennis Court 1
2,GUEST GUEST,Tennis Court 2
3,GUEST GUEST,Tennis Court 1
4,GUEST GUEST,Tennis Court 1
...,...,...
792,GUEST GUEST,Tennis Court 2
793,Charles Owen,Tennis Court 2
794,David Jones,Tennis Court 2
795,Charles Owen,Tennis Court 2


### Q8: <a id = 'q8'></a>
Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [10]:
try:
    con = engine.connect()
    rs = con.execute('''
        SELECT firstname || ' ' || surname AS person
            , CASE WHEN Bookings.memid > 0 THEN membercost
              WHEN Bookings.memid = 0 THEN guestcost END AS cost
            , DATE(joindate) AS date
        FROM Bookings
        INNER JOIN Facilities
            ON Bookings.facid = Facilities.facid
        INNER JOIN Members
            ON Bookings.memid = Members.memid
        WHERE DATE(joindate) = '2012-09-14'
            AND cost > 30
    ''')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    con.close()
    display(df)
except:
    print("Unfortunately, the information you look for does not exist.")

Unfortunately, the information you look for does not exist.


### Q9: <a id = 'q9'></a>
This time, produce the same result as in Q8, but using a subquery.

In [11]:
try:
    con = engine.connect()
    rs = con.execute('''
        SELECT firstname
            , (SELECT CASE WHEN memid = 0 THEN guestcost
                      ELSE membercost END AS cost
               FROM Facilities) AS cost
            , (SELECT DATE(joindate)
               FROM Facilities) AS joindate
        FROM Members
        WHERE (SELECT DATE(joindate)
               FROM Facilities) = '2012-09-14'
              AND cost > 30
    ''')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    con.close()
    display(df)
except:
    print('The subquery produces the same result.')

The subquery produces the same result.


### Part 2 <a id = 'part2'></a>

## Q10: <a id = 'q10'></a>
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [12]:
con = engine.connect()
rs = con.execute('''
    WITH dataf AS (SELECT name
            , SUM(CASE WHEN Bookings.memid = 0 THEN guestcost*slots
              ELSE membercost*slots END) AS revenue
            , guestcost
            , membercost
            , SUM(CASE WHEN Bookings.memid = 0 THEN 1
              ELSE 0 END) AS guests
            , SUM(CASE WHEN Bookings.memid > 0 THEN 1
              ELSE 0 END) AS members
            , SUM(CASE WHEN Bookings.memid = 0 THEN slots
              ELSE 0 END) AS guests_half_hourly_rate
            , SUM(CASE WHEN Bookings.memid > 0 THEN slots
              ELSE 0 END) AS members_half_hourly_rate
        FROM Bookings
        INNER JOIN Members
            ON Members.memid = Bookings.memid
        INNER JOIN Facilities
            ON Facilities.facid = Bookings.facid 
        GROUP BY name
        ORDER BY revenue DESC)
    
    SELECT *
    FROM dataf
    WHERE revenue < 1000
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
display(df)

,name,revenue,guestcost,membercost,guests,members,guests_half_hourly_rate,members_half_hourly_rate
0,Pool Table,270,5,0,53,783,54,856
1,Snooker Table,240,5,0,23,421,48,860
2,Table Tennis,180,5,0,18,385,36,794


## Q11: <a id='q11'></a>
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [13]:
con = engine.connect()
rs = con.execute('''
    WITH neatable AS (SELECT memid
            , firstname || ' ' || surname AS fullname
            , CAST(recommendedby AS INT) AS recommended_by
        FROM Members)
    
    SELECT a.memid
        , a.fullname AS members
        , a.recommended_by AS rec_memid
        , b.fullname AS recommenders
    FROM neatable AS a
    INNER JOIN neatable AS b
        ON a.recommended_by = b.memid
    WHERE a.recommended_by > 0
    ORDER BY members
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
display(df)

,memid,members,rec_memid,recommenders
0,21,Anna Mackenzie,1,Darren Smith
1,12,Anne Baker,9,Ponder Stibbons
2,10,Charles Owen,1,Darren Smith
3,11,David Jones,4,Janice Joplette
4,17,David Pinker,13,Jemima Farrell
5,26,Douglas Jones,11,David Jones
6,36,Erica Crumpet,2,Tracy Smith
7,15,Florence Bader,9,Ponder Stibbons
8,5,Gerald Butters,1,Darren Smith
9,27,Henrietta Rumney,20,Matthew Genting


## Q12: <a id = 'q12'></a>
Find the facilities with their usage by member, but not guests

In [14]:
con = engine.connect()
rs = con.execute('''
    WITH dataf AS (SELECT name
            , SUM(CASE WHEN Bookings.memid > 0 THEN 1
              ELSE 0 END) AS members
        FROM Bookings
        INNER JOIN Members
            ON Members.memid = Bookings.memid
        INNER JOIN Facilities
            ON Facilities.facid = Bookings.facid 
        GROUP BY name
        ORDER BY members DESC)
    
    SELECT *
    FROM dataf
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
display(df)

,name,members
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


## Q13: <a id = 'q13'></a>
Find the facilities usage by month, but not guests

In [15]:
con = engine.connect()
rs = con.execute('''
    WITH dataf AS (SELECT name
            , SUM(CASE WHEN Bookings.memid > 0 THEN 1
              ELSE 0 END) AS members
            , STRFTIME('%m', joindate) AS Month
        FROM Bookings
        INNER JOIN Members
            ON Members.memid = Bookings.memid
        INNER JOIN Facilities
            ON Facilities.facid = Bookings.facid 
        GROUP BY name
        ORDER BY members DESC)
    
    SELECT *
    FROM dataf
    ''')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
display(df)

,name,members,Month
0,Pool Table,783,07
1,Snooker Table,421,07
2,Massage Room 1,421,07
3,Table Tennis,385,07
4,Badminton Court,344,07
5,Tennis Court 1,308,07
6,Tennis Court 2,276,07
7,Squash Court,195,07
8,Massage Room 2,27,07
